# Convoluted Neural Network: 1st Model Trial

## Imports

In [1]:
import numpy as np
import pandas as pd

## To be reconsidered at later date (RNN)

### Recurrent Neural Network Prep (to be reconsidered)

#### Sort dataframe by period !

First df needs to be sorted by period!

In [2]:
# sorted_df = df.sort_values(by = ['period'])

#### Train & Test Dataframe

In [3]:
# len_ = int(0.8 * sorted_df.shape[0])

In [4]:
# df_train = sorted_df[:len_]
# df_test = sorted_df[len_:]

#### Turning df_train and df_test into Gaussian 3D arrays

In [5]:
# img3D_conv = gaussian_filter(img_list, sigma=(2,2,2))

#### Splitting into sub-samples

**Decide how many observations (sample number) you need for a chosen observation 'length' (no. of images / hours)**

Sample the df as many times as the no. of observations indicated and split each into X and y.


get_X_y should call the split_subsample function, which in turn calls the subsample_sequence function.

i.e. first sample, then split, then store in X and y arrays


X_train and y_train should be extracted from the train dataframe (vice versa for X_test and y_test)!

## Dummie Model

### X and y

In [ ]:
X_ = np.random.randint(2, size = (50, 24, 1_000, 1_000)).astype(np.int32)

In [90]:
#y = np.array([np.random.randint(2, size = (3, 1_000, 1_000), dtype = int).flatten() for k in range(50)])

In [106]:
X = np.empty((50, 24, 1_000, 1_000))

y = np.array([np.empty((24, 1_000, 1_000), dtype = int).flatten() for k in range(50)])

In [107]:
print(X.shape)
print(y.shape)

(50, 24, 1000, 1000)
(50, 24000000)


### Train, test, split

In [108]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [109]:
X_train.shape

(40, 24, 1000, 1000)

In [110]:
X_test.shape

(10, 24, 1000, 1000)

## CNN

In [111]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [112]:
from tensorflow.keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(mean_squared_error(y_true, y_pred))

In [113]:
model = models.Sequential()

In [114]:
tf.keras.layers.GaussianNoise(stddev = root_mean_squared_error, input_shape = (24, 1_000, 1_000))

In [115]:
model.add(layers.Conv2D(32, kernel_size = 3, activation = 'relu'))

In [116]:
model.add(layers.Flatten())

In [117]:
model.add(layers.Dense(3 * 1_000 * 1_000))

In [118]:
model.compile(loss = 'mse',
                  optimizer = 'adam',
                  metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [15]:
def first_model():
    
    model = models.Sequential() # initialize model

    # input shape 10 as 10 observations
    
    tf.keras.layers.GaussianNoise(stddev, **kwargs)
    
    model.add(layers.Conv2D(32, kernel_size = 3, activation = 'relu', input_shape = (24, 1_000, 1_000))) 

    model.add(layers.Conv2D(16, kernel_size = 3))

    model.add(layers.Conv2D(32, kernel_size = 6))

    model.add(layers.Flatten())

    model.add(layers.Dense(1_000, 1_000, 50)

    #model.compile(loss='mse',
                  #optimizer = 'adam',
                  #metrics=['accuracy'])

    return model
    
model = first_model()

# prevent overfitting with early stopping
# es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, verbose=1, restore_best_weights=True)

SyntaxError: invalid syntax (<ipython-input-15-7f9f0048a7e2>, line 20)

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(mean_squared_error(y_true, y_pred))

model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(28,28,5)))
model.add(Flatten())
model.add(Dense(28*28*3))

model.compile(optimizer = "rmsprop", loss = root_mean_squared_error, metrics =["accuracy"])

X_other = np.empty((1, 28, 28, 5)) # 1 3d array with zeros
y_other = np.array([np.empty((28, 28, 3), dtype=int).flatten()])

model.fit(X_other, y_other)